In [11]:
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.datasets import make_regression
import pandas as pd
import numpy as np
import seaborn as sns

## Dataset
<div>
<img src="https://dataingovernment.blog.gov.uk/wp-content/uploads/sites/46/2020/08/synthetic_data_image.png" width="500"/>
</div>
Dalam mempelajari machine learning, kita dapat membentuk dataset yang dibentuk sesuka hati dengan mudah. Dalam masalah regresi, fungsi yang dapat digunakan adalah make_regression (pada klasifikasi make_classification). Argumen utama yang digunakan adalah n_samples (banyaknya baris/observasi) dan n_features (banyaknya kolom prediktor/features). Argumen tambahan yang bisa digunakan meliputi noise (menambah gaussian noise agar data lebih acak), random_state (agar reproducible), dan n_informative.

Tidak hanya regresi dan klasifikasi, skelarn juga mendukung pembuatan dataset untuk clustering dan analisis lainnya. Informasi mengenai datasets dapat ditemukan pada: 

https://scikit-learn.org/stable/datasets.html 

### Generate regression data

Di bawah ini adalah salah satu contoh membentuk dataset untuk keperluan regresi. Beberapa tahapan tambahan juga dilakukan agar dataset yang terbentuk lebih menarik untuk di analisis.

- Gunakan fungsi make_regression dengan jumlah rows dan features yang dirasa cukup.
- Ubah output yang dihasilkan ke dalam bentuk dataframe agar mudah untuk dilihat.
- Secara default, data yang dihasilkan akan berada di sekitar 0. Untuk menambah kerumitan data, kita akan melakukan beberapa transformasi sederhana.
- Kita akan melakukan binning sederhana terhadap beberapa kolomnya untuk merubahnya menjadi variabel kategorik. Selain itu, kita akan merubah scale dari beberapa kolom menjadi ratusan atau ribuan. Bentuk transformasi lainnya seperti log juga diterapkan.

##### Make Regression

In [3]:
X, y = make_regression(n_samples = 1675, n_features = 11, n_informative = 3, random_state = 10969, noise = 3.4)

##### Ubah ke DataFrame agar familiar

In [4]:
colnames = ['Field'+str(i) for i in range(1, len(X.T)+1)]

df = pd.DataFrame(X, columns = colnames)
df['Label'] = y
df.head()

,Field1,Field2,Field3,Field4,Field5,Field6,Field7,Field8,Field9,Field10,Field11,Label
0,1.128841,-1.276751,-2.724161,-1.229286,-0.141915,-0.737333,-0.844138,-0.038986,-0.631474,-0.728941,0.641624,-78.911166
1,-2.691635,-1.509216,-1.577358,-0.769756,0.123455,-0.627568,0.193674,0.710674,-0.001338,0.069663,0.436281,65.740236
2,-0.341965,-1.329386,-1.155901,0.146192,-0.116123,0.348146,-0.845446,-0.277405,0.101198,-0.089031,0.380754,-94.575471
3,0.470145,-0.809076,0.231595,0.312315,-0.248064,0.067386,-0.717591,-0.419167,0.662053,-0.806093,-0.662331,-88.542039
4,0.234980,-0.404053,1.489868,0.777949,-2.271965,0.240064,-0.123021,-0.530795,0.082038,-0.007872,-1.458746,-42.210259


##### Lakukan transformasi untuk menambah kerumitan data

In [6]:
# Ubah Field2 menjadi bentuk log
df['Field2'] = np.log(np.abs(df['Field2']))

# Ubah Field3 menjadi kategorik dengan 2 kategori (biner)
df['Field3'] = [1 if i < 0 else 0 for i in df['Field3']]

# Ubah Field4 menjadi kategorik dengan 4 kategori
df['Field4'] = [0 if i < -1 else 1 if i < 0 else 2 if i < 1 else 3 for i in df['Field4']]

# Ubah Field7 dengan rumus aneh 
df['Field7'] = np.sqrt(np.abs(df['Field7'])) * np.pi ** 3

# Ubah Field 8 menjadi ratusan
df['Field8'] = df['Field8'] * 134

# Ubah Field 9
df['Field9'] = df['Field1'] * 2 - df['Field11']*np.pi

# Ubah Field 10
df['Field10'] = np.abs(df['Field10'])

df.head()

,Field1,Field2,Field3,Field4,Field5,Field6,Field7,Field8,Field9,Field10,Field11,Label
0,1.128841,0.244319,1,0,-0.141915,-0.737333,28.487627,-5.224114,0.241962,0.728941,0.641624,-78.911166
1,-2.691635,0.411591,1,1,0.123455,-0.627568,13.645363,95.230376,-6.753886,0.069663,0.436281,65.740236
2,-0.341965,0.284717,1,2,-0.116123,0.348146,28.509692,-37.172257,-1.880105,0.089031,0.380754,-94.575471
3,0.470145,-0.211863,0,2,-0.248064,0.067386,26.265653,-56.168399,3.021064,0.806093,-0.662331,-88.542039
4,0.234980,-0.906209,0,2,-2.271965,0.240064,10.875265,-71.126485,5.052746,0.007872,-1.458746,-42.210259


### Split Train Test

In [9]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['Label'], axis = 1), df['Label'], test_size = 0.3456789)

## Modelling 

### Baseline

Ini adalah best practice, atau sangat disarankan untuk membentuk sebuah model baseline.

Pada dasarnya, model baseline digunakan sebagai pembanding model-model lainnya yang nanti akan dibentuk. Apabila model baru lebih buruk performanya dibandingkan baseline, boleh jadi menunjukkan bahwa model baru tersebut bukan merupakan improvement.

Model baseline juga dapat dibentuk sebelum preprocessing (menggunakan data mentah).

### Algoritma

Kita akan membandingkan 2 model regresi, yaitu Adaptive Boosting (AdaBoost) dan Orthogonal Matching Pursuit (OMP).

Ukuran metrics evaluasi yang digunakan adalah Mean Absolute Error dan Mean Square Log Error.

Penggunakan Mean Square Log Error lebih robust dibandingkan MSE biasa. Apabila terdapat data test yang bersifat outlier, bisa jadi nilai prediksinya sangat besar sekali. Apabila ternyata prediksinya meleset, maka errornya juga akan besar (bahkan menutupi error2 kecil dari observasi lainnya). Namun, dataset yang dimiliki kita, nilai Label ada yang negatif (sehingga tidak bisa di Log). Alternatif lainnya adalah dengan melihat ukuran evaluasi yang robust terhadap outlier, seperti median squared error.

Pada dasarnya, dalam sklearn tidak ada metrics bernama median squared error. Oleh karenanya, kita perlu untuk membentuk sebuah fungsi yang dapat menghitung nilai tersebut. Kita dapat menggunakan metrics buatan sendiri (seaneh apapun itu).

### Cross Validation

Kedua model akan dikenakan cross validation sederhana berupa KFold dengan Fold = 5. 

<div>
<img src='https://scikit-learn.org/stable/_images/grid_search_cross_validation.png', width = 500>
</div>

In [10]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import OrthogonalMatchingPursuit

##### Bentuk Fungsi Median Square Error

In [48]:
def median_square_error(y_true, y_pred):
  return(np.square(np.subtract(y_true, y_pred)).median())

##### Baseline

Dari hasil di bawah, AdaBoost terlihat lebih buruk dibandingkan OMP, baik diukur berdasarkan MAE, MSE, maupun Median SE.

Bagaimana apabila model kita dihadapkan pada data yang baru? Apakah performanya akan sama?

In [27]:
# Baseline AdaBoost
ada = AdaBoostRegressor(random_state = 887, n_estimators = 100, learning_rate = 0.01)
ada.fit(x_train, y_train)
ada_predict = ada.predict(x_test)
ada_MAE = mean_absolute_error(y_test, ada_predict)
ada_MSE = mean_squared_error(y_test, ada_predict)
ada_MedSE = median_square_error(y_test, ada_predict)

# Baseline OMP
omp = OrthogonalMatchingPursuit()
omp.fit(x_train, y_train)
omp_predict = omp.predict(x_test)
omp_MAE = mean_absolute_error(y_test, omp_predict)
omp_MSE = mean_squared_error(y_test, omp_predict)
omp_MedSE = median_square_error(y_test, omp_predict)

# Hasil
result = pd.DataFrame({
    'Nama model' : ['Ada', 'OMP'],
    'MAE' : [ada_MAE, omp_MAE],
    'MSE' : [ada_MSE, omp_MSE],
    'MedSE' : [ada_MedSE, omp_MedSE]
})

result


,Nama model,MAE,MSE,MedSE
0,Ada,75.534593,9036.381672,3725.023494
1,OMP,71.403068,7947.099775,3431.818740


##### Cross validation

Asumsikan bahwa kita ingin menggunakan MAE, MSE, dan MedSE sebagai ukuran evaluasi cross validation sekaligus (lebih efisien jika dibandingkan menghitung satu per satu). Untuk itu, kita perlu membentuk sebuah scorer. Simpelnya, scorer adalah instruksi sederhana yang berisi daftar metrics yang kita inginkan.

Terdapat beberapa informasi yang bisa diperoleh dari menggunakan cross validation.
- fit_time, yaitu waktu yang diperlukan untuk fitting model pada setiap 'fold'. Semakin rendah maka semakin baik. Umumnya kita ingin melihat nilai mean, deviasi, dan maximumnya.
- score_time, yaitu waktu untuk menghitung score metrics. Umumnya sangat kecil.
- test_*metrics*, yaitu ukuran evaluasi pada data yang digunakan sebagai test pada CV.


In [69]:
my_cv = KFold(8)

my_scorer = {
    'MAE' : 'neg_mean_absolute_error',
    'Median Squared Error' : make_scorer(median_square_error, greater_is_better = False),
}

ada_cv = cross_validate(ada, x_train, y_train, scoring = my_scorer, cv = my_cv)
omp_cv = cross_validate(omp, x_train, y_train, scoring = my_scorer, cv = my_cv)

In [70]:
result_cv = pd.DataFrame({
    'Model' : ['Ada', 'OMP'],
    'Avg fit time' : [np.mean(ada_cv['fit_time']), np.mean(omp_cv['fit_time'])],
    'Sd fit time' : [np.std(ada_cv['fit_time']), np.std(omp_cv['fit_time'])], 
    'Max fit time' : [np.max(ada_cv['fit_time']), np.max(omp_cv['fit_time'])],
    '|' : ['|', '|'],
    'Avg MAE' : [np.mean(ada_cv['test_MAE']), np.mean(omp_cv['test_MAE'])],
    'Sd MAE' : [np.std(ada_cv['test_MAE']), np.std(omp_cv['test_MAE'])],
    'Max MAE' : [np.max(ada_cv['test_MAE']), np.max(omp_cv['test_MAE'])],
    '||' : ['||', '||'],
    'Avg MedSE' : [np.mean(ada_cv['test_Median Squared Error']), np.mean(omp_cv['test_Median Squared Error'])],
    'Sd MedSE' :  [np.std(ada_cv['test_Median Squared Error']), np.std(omp_cv['test_Median Squared Error'])],
    'Max MedSE' :  [np.max(ada_cv['test_Median Squared Error']), np.max(omp_cv['test_Median Squared Error'])]
})

result_cv

,Model,Avg fit time,Sd fit time,Max fit time,|,Avg MAE,Sd MAE,Max MAE,||,Avg MedSE,Sd MedSE,Max MedSE
0,Ada,0.259131,0.009349,0.281001,|,-65.090663,7.305644,-56.205678,||,-3143.146408,946.706369,-1980.609141
1,OMP,0.002611,0.000183,0.003012,|,-66.395133,12.032973,-52.091456,||,-3349.059091,1888.627169,-1788.169701


Abaikan nilai negatif di atas (hal ini dikarenakan scoring function skelarn didasari pada loss function).

Apabila kita perhatikan, model OMP memiliki waktu fitting yang jauh lebih cepat dibandingkan AdaBoost. Akan tetapi, nilai rata2 dan deviasi MAE dari AdaBoost lebih kecil dibandingkan OMP, menunjukkan bahwa model ini lebih konsisten secara umum.